Part2: 
First the helper functions

In [ ]:
Using namespace System.Collections.Generic

function Get-PSObjectProperties {
    <#
    .synopsis
        iterate/enumerate an object's properties  [zero dependency]
    .example
        PS> gi . | IterProp
    .example
        PS> (get-date) | iterProp | Ft -AutoSize MemberType, Name, Value, TypeNameOfValue
    #>
    [Alias('IterProp')]
    [cmdletbinding()]
    param(
        # any object to enumerate
        [Parameter(Mandatory, ValueFromPipeline)]
        [object]$InputObject
    )
    begin {}
    process {
        $InputObject.PSObject.Properties
    }
    end {}
}

function Get-ObjectTypeInfo {
    <#
    .synopsis
        iterate/enumerate an object's properties  [zero dependency]
    .example
        PS> gi . | IterProp
    .example
        gi . | IterProp | ft Name, TypeNameOfValue, Value -AutoSize
    .example
        PS> (get-date) | iterProp | Ft -AutoSize MemberType, Name, Value, TypeNameOfValue
    #>
    [Alias('TypeInfo')]
    [cmdletbinding()]
    param(
        # any object to enumerate
        [AllowNull()]
        [Parameter(Mandatory, ValueFromPipeline)]
        [object]$InputObject
    )
    begin {}
    process {
        $meta = @{ }
        if ($null -eq $InputObject) {
            return 'True Null'
        }
        $firstElement = if ($InputObject.count -gt 0) {
            $InputObject[0].GetType().Fullname
        } else { $null }


        $itemCount = $InputObject.Count
        $tinfo = $InputObject.GetType()
        $meta = [ordered]@{
            PSTypeName     = 'nin.TypeInfo'
            Type           = $Tinfo.Name
            BaseType       = $tinfo.BaseType.FullName
            HasElements    = $InputObject.Count -gt 0
            FirstElement   = $firstElement
            Count          = $InputObject.Count
            Length         = $InputObject.Length # note: can be misleading
            PSTypeNameList = $InputObject.PSTypeNames
            Namespace      = $Tinfo.Namespace
            FullName       = $tinfo.FullName
        }

        if ($InputObject -is 'String') {
            $meta += @{
                StringLength  = $InputObject.Length
                IsWhiteSpace  = [string]::IsNullOrWhiteSpace( $InputObject )
                IsNullOrEmpty = [string]::IsNullOrEmpty( $InputObject )
                IsAString     = $InputObject -is 'String'
                AsTypeName    = $InputObject -as 'type'
            }
        }

        return [pscustomObject]$Meta
    }
    end {}
}

In [ ]:
# Samples to inspect
$nums = 0..4
$letters = 'a'..'e'
$target = Get-Item .
$envVars = ls env:
$now = get-date
$folder = gi 'c:\'
$items = ls 'c:\' -depth 1


In [ ]:
# now lets have fun
$target
| IterProp
| Join-String -sep "`n- " -op "`n- "

# $now
# | IterProp
# | Join-String -sep "`n" -op "`n" { $_.Name, "`n`t", $_.Type, "`n`t", $_.value -join ' = ' }



- string PSPath=Microsoft.PowerShell.Core\FileSystem::C:\Users\cppmo_000\SkyDrive\Documents\2022\My_Github\notebooks\Pwsh\Objects
- string PSParentPath=Microsoft.PowerShell.Core\FileSystem::C:\Users\cppmo_000\SkyDrive\Documents\2022\My_Github\notebooks\Pwsh
- string PSChildName=Objects
- PSDriveInfo PSDrive=C
- ProviderInfo PSProvider=Microsoft.PowerShell.Core\FileSystem
- bool PSIsContainer=True
- System.String Mode{get=Mode;}
- System.String ModeWithoutHardLink{get=ModeWithoutHardLink;}
- System.Object BaseName {get=$this.Name;}
- Target = LinkTarget
- System.String LinkType{get=GetLinkType;}
- System.IO.DirectoryInfo Parent {get;}
- System.IO.DirectoryInfo Root {get;}
- string FullName {get;}
- string Extension {get;}
- string Name {get;}
- bool Exists {get;}
- datetime CreationTime {get;set;}
- datetime CreationTimeUtc {get;set;}
- datetime LastAccessTime {get;set;}
- datetime LastAccessTimeUtc {get;set;}
- datetime LastWriteTime {get;set;}
- datetime LastWriteTimeUtc {get;set;}
-

In [ ]:
# Using 'PadRight/Left' both [1] aligns our strings, and, [2], ensures substring is does not have an index error
$Colors = @{
    TypeName = $PSStyle.Foreground.FromRgb('#4974a1')
    MemberName = $PSStyle.Foreground.FromRgb('#a4dcff')
    MemberValue = $PSStyle.Foreground.FromRgb('#dbdc9a')    
}
$target | IterProp | Join-String -sep "`n" -op "`n" { @(        
        $max = 20
        $Colors.TypeName
        $_.TypeNameOfValue.ToString().padright($max).substring(0, $max)
        ' '
        $Colors.MemberName
        $_.name.ToString().Padright( $max).Substring(0, $max)
        ' '        
        $Colors.MemberValue
        $_.Value.ToString().padright($max+20).substring(0, $max+20)
        ' '
    ) -join ''
}


System.String        PSPath               Microsoft.PowerShell.Core\FileSystem::C: 
System.String        PSParentPath         Microsoft.PowerShell.Core\FileSystem::C: 
System.String        PSChildName          Objects                                  
System.Management.Au PSDrive              C                                        
System.Management.Au PSProvider           Microsoft.PowerShell.Core\FileSystem     
System.Boolean       PSIsContainer        True                                     
System.String        Mode                 la---                                    
System.String        ModeWithoutHardLink  la---                                    
System.Object        BaseName             Objects                                  


System.IO.DirectoryI Parent               C:\Users\cppmo_000\SkyDrive\Documents\20 
System.IO.DirectoryI Root                 C:\                                      
System.String        FullName             C:\Users\cppmo_000\SkyDrive\Doc

In [ ]:
# $target | IterProp | Join-String -sep "`n- " -op "`n- "

$target | IterProp
| Format-Table Name, TypeNameOfValue, Value -AutoSize

$now | iterProp
| Format-Table -AutoSize MemberType, Name, Value, TypeNameOfValue




Name                TypeNameOfValue                           Value
----                ---------------                           -----
PSPath              System.String                             Microsoft.PowerShell.Core\FileSystem…
PSParentPath        System.String                             Microsoft.PowerShell.Core\FileSystem…
PSChildName         System.String                             Objects
PSDrive             System.Management.Automation.PSDriveInfo  C
PSProvider          System.Management.Automation.ProviderInfo Microsoft.PowerShell.Core\FileSystem
PSIsContainer       System.Boolean                            True
Mode                System.String                             la---
ModeWithoutHardLink System.String                             la---
BaseName            System.Object                             Objects
Target              System.String                             
LinkType            System.String                             
Parent              System.IO.D


PSPath              Microsoft.PowerShell
PSParentPath        Microsoft.PowerShell
PSChildName         Objects             
PSDrive             C                   
PSProvider          Microsoft.PowerShell
PSIsContainer       True                
Mode                la---               
ModeWithoutHardLink la---               
BaseName            Objects             


Parent              C:\Users\cppmo_000\S
Root                C:\                 
FullName            C:\Users\cppmo_000\S
Extension                               
Name                Objects             
Exists              True                
CreationTime        7/21/2022 8:22:09 PM
CreationTimeUtc     7/22/2022 1:22:09 AM
LastAccessTime      8/15/2022 12:13:41 P
LastAccessTimeUtc   8/15/2022 5:13:41 PM
LastWriteTime       8/15/2022 12:11:17 P
LastWriteTimeUtc    8/15/2022 5:11:17 PM

Attributes          Directory, Archive, 
